In [1]:
#SIMULATION - TREATMENT MAP
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Module conducts the following analysis:

# ---- Extended Data Figure 2: Spatial distribution of offtakers targeted by spatially targeted CfD regime

# Module is input for:

# ---- NA

In [ ]:
#SET-UP
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.getenv("PROJ_LIB")

check_and_load <- function(packages) {
  for (pkg in packages) {
    if (!requireNamespace(pkg, quietly = TRUE)) {
      message(paste("Installing missing package:", pkg))
      install.packages(pkg, dependencies = TRUE, repos = "https://cloud.r-project.org")
    }
    if (!(pkg %in% (.packages()))) {
      suppressPackageStartupMessages(library(pkg, character.only = TRUE))
    }
  }
}

required_packages <- c(
  "sf", "dplyr", "purrr", 
  "forcats","tidyr", "ggplot2", "rnaturalearth", "rnaturalearthdata"
)

check_and_load(required_packages)

# ── LOAD DATA --------------------------------------------------------------
results_all <- readRDS("results_all.rds")
offtakers   <- readRDS("offtakers.rds") 

In [ ]:
#DATAFILES
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
results_all <- readRDS("results_all.rds")
offtakers   <- readRDS("offtakers.rds") 

world <- ne_countries(scale = "medium", returnclass = "sf")

In [ ]:
# load geometry
offtakers_sf <- offtakers %>%
  st_transform(4326) %>%
  select(plant_id, geometry)

# map limits

eu_xlim <- c(-15, 45)
eu_ylim <- c(30, 75)

# centrality types present
centrality_types <- c("betweenness", "degree")
interventions    <- c("high", "random", "low")

facet_spec <- expand.grid(
  centrality  = centrality_types,
  intervention = interventions,
  stringsAsFactors = FALSE
) %>%
  mutate(key = paste(centrality, intervention, sep = "_"))

# Extract treated agents
treated_long <- map_dfr(seq_len(nrow(facet_spec)), function(i) {
  this_key <- facet_spec$key[i]
  this_cent <- facet_spec$centrality[i]
  this_int  <- facet_spec$intervention[i]

  ta <- results_all[[this_key]]$treated_offtakers
  if (is.null(ta)) return(NULL)

  ta %>%
    mutate(
      centrality_type = this_cent,
      intervention    = this_int
    )
})

# Mege gometry
map_df <- offtakers_sf %>%
  left_join(treated_long, by = "plant_id") %>%
  mutate(
    treated_flag = treatment != "Untreated",

    centrality_type = factor(centrality_type),
    centrality_type = fct_recode(
      centrality_type,
      "Betweenness" = "betweenness",
      "Degree"      = "degree"
    ),
    centrality_type = fct_relevel(centrality_type, "Degree", "Betweenness"),

    intervention = factor(intervention),
    intervention = fct_relevel(
      fct_recode(
        intervention,
        "High Centrality"   = "high",
        "Random" = "random",
        "Low Centrality"    = "low"
      ),
      "High", "Random", "Low"
    )
  ) %>%
  st_as_sf()

# Theme
map_theme <- theme_classic(base_size = 22) +
  theme(
    axis.text.x  = element_blank(),
    axis.text.y  = element_blank(),
    axis.ticks   = element_blank(),
    axis.title.x = element_blank(),
    axis.title.y = element_blank(),

    strip.placement = "outside",
    strip.text.y.left = element_text(angle = 0, hjust = 1, vjust = 0.5),

    # Panels
    panel.border = element_rect(color="black", fill=NA, linewidth=0.6),
    panel.spacing = unit(0.2, "cm"),

    # Title/legend
    legend.position = "none",
    strip.background = element_blank(),
    plot.title = element_text(size = 24) 
  )

# Plot
extended_data_figure_2 <- ggplot() +
  # basemap
  geom_sf(data = world, fill="grey97", color="grey85", size=0.2) +

  # all plants (background)
  geom_sf(
    data = map_df,
    color = "grey80", size = 0.1, alpha = 0.6
  ) +

  # treated plants (highlight)
  geom_sf(
    data = filter(map_df, treated_flag),
    color = "#4DBBD5FF", size = 0.5, alpha = 1
  ) +

  facet_grid(
    rows = vars(centrality_type),
    cols = vars(intervention),
    switch = "y"
  ) +

  coord_sf(
    xlim = eu_xlim,
    ylim = eu_ylim,
    expand = FALSE
  ) +

  labs(
    title = "Spatial Distribution of Offtakers subject to CfD regime by Centrality and Intervention Target"
  ) +

  map_theme


options(repr.plot.width = 18, repr.plot.height = 14, repr.plot.res = 600)

extended_data_figure_2 

# Note that this figure is saved as PNG since saving as PDF distorts the visualization of points
ggsave("extended-data-figure-2 .png", extended_data_figure_2 ,device="png", width=24, height=12,units="in", dpi=600, limitsize=FALSE)